In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.15.0'

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('C:\\Users\\Abhinav Raj\\Downloads\\Data sets\\hindi_lyrics.csv')

In [6]:
df

,SongID,Lyrics
0,16,\n\n--Male--\nAaaaa Aaaaa\n--Female--\nAaaaaa\...
1,17,\n\n--Male1--\nAnkhiyon Hi Ankhiyon Mein\nRati...
2,18,"\n\n--Male1--\nAray Peeli Chotiyaan,\nHawaeyn ..."
3,19,\n\nPyar Itna Na Kar\nYeh Dil Jaata Hai Bhar\n...
4,20,Zaraa Maara Maara Sa\nJaane Kyun Dil Ye Ban Ba...
...,...,...
1286,1581,\n\nKaara fankaara kab aaye re\nKaara fankaar...
1287,1582,\n\nZameen-o-aasmaan ne kya baat ki hai\nGira...
1288,1583,\n\nMaula Wa Sallim wassalim da-iman abadan\n...
1289,1584,\n\nBhavara\nRe ga re ga re ga re ga pa ma ga...


In [7]:
df.shape[0]

1291

In [8]:
len(df['artist'].unique())

KeyError: 'artist'

In [ ]:
df['artist'].value_counts()[:10]

In [11]:
data = ','.join(df['Lyrics'])

In [12]:
data[:369]

'\n\n--Male--\nAaaaa Aaaaa\n--Female--\nAaaaaa\nAaaaaa\n--Male--\nMeetha Sa Ishq Laage\nKadvi Judayi\nYaar Mera Sacha Laage\nJhooti Khudayi\nMeetha Sa Ishq Laage\nKadvi Judayi\nYaar Mera Sacha Laage\nJhooti Khudayi\nChandni Ne Tan Pe Mere\nChaadar Bhichayee\nOdha Jo Tune Mujhko\nSaans Laut Aayi\nChandni Ne Tan Pe Mere\nChaadar Bhichayee\nOdha Jo Tune Mujhko\nSaans Laut Aayi\nOhooooo Ohhoooo\n'

In [13]:
chars = sorted(list(set(data)))

In [14]:
vocab_size =len(chars)
vocab_size

89

In [15]:
char_to_ix = {ch: i for i, ch in enumerate(chars)}

In [16]:
ix_to_char = {i: ch for i, ch in enumerate(chars)}

In [17]:
def one_hot_encoder(index):
    return np.eye(vocab_size)[index]

In [18]:
hidden_size = 100

In [19]:
seq_length = 25

In [20]:
learning_rate = 1e-1

In [21]:
seed_value = 42
tf.set_random_seed(seed_value)
random.seed(seed_value)

In [22]:
inputs = tf.placeholder(shape=[None, vocab_size],dtype=tf.float32, name="inputs")
targets = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="targets")

In [23]:
init_state = tf.placeholder(shape=[1, hidden_size], dtype=tf.float32, name="state")

In [24]:
initializer = tf.random_normal_initializer(stddev=0.1)

In [25]:
tf
with tf.variable_scope("RNN") as scope:
    h_t = init_state
    y_hat = []

    for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0:
            scope.reuse_variables()  

        #input to hidden layer weights
        U = tf.get_variable("U", [vocab_size, hidden_size], initializer=initializer)

        #hidden to hidden layer weights
        W = tf.get_variable("W", [hidden_size, hidden_size], initializer=initializer)

        #output to hidden layer weights
        V = tf.get_variable("V", [hidden_size, vocab_size], initializer=initializer)

        #bias for hidden layer
        bh = tf.get_variable("bh", [hidden_size], initializer=initializer)

        #bias for output layer
        by = tf.get_variable("by", [vocab_size], initializer=initializer)

        h_t = tf.tanh(tf.matmul(x_t, U) + tf.matmul(h_t, W) + bh)

        y_hat_t = tf.matmul(h_t, V) + by

        y_hat.append(y_hat_t)       

In [26]:
output_softmax = tf.nn.softmax(y_hat[-1])  
outputs = tf.concat(y_hat, axis=0)

In [27]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))

In [28]:
hprev = h_t

In [29]:
minimizer = tf.train.AdamOptimizer()

In [30]:
gradients = minimizer.compute_gradients(loss)

In [31]:
threshold = tf.constant(5.0, name="grad_clipping")

In [32]:
clipped_gradients = []
for grad, var in gradients:
    clipped_grad = tf.clip_by_value(grad, -threshold, threshold)
    clipped_gradients.append((clipped_grad, var))

In [33]:
updated_gradients = minimizer.apply_gradients(clipped_gradients)

In [34]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [35]:
pointer = 0
iteration = 0

In [ ]:
while True:
    
    if pointer + seq_length+1 >= len(data) or iteration == 0:
        hprev_val = np.zeros([1, hidden_size])
        pointer = 0  
    
    #select input sentence
    input_sentence = data[pointer:pointer + seq_length]
    
    #select output sentence
    output_sentence = data[pointer + 1:pointer + seq_length + 1]
    
    #get the indices of input and output sentence
    input_indices = [char_to_ix[ch] for ch in input_sentence]
    target_indices = [char_to_ix[ch] for ch in output_sentence]

    #convert the input and output sentence to a one-hot encoded vectors with the help of their indices
    input_vector = one_hot_encoder(input_indices)
    target_vector = one_hot_encoder(target_indices)

    
    #train the network and get the final hidden state
    hprev_val, loss_val, _ = sess.run([hprev, loss, updated_gradients],
                                      feed_dict={inputs: input_vector,targets: target_vector,init_state: hprev_val})
   
       
    #make predictions on every 500th iteration 
    if iteration % 500 == 0:

        #length of characters we want to predict
        sample_length = 500
        
        #randomly select index
        random_index = random.randint(0, len(data) - seq_length)
        
        #sample the input sentence with the randomly selected index
        sample_input_sent = data[random_index:random_index + seq_length]
    
        #get the indices of the sampled input sentence
        sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]
        
        #store the final hidden state in sample_prev_state_val
        sample_prev_state_val = np.copy(hprev_val)
        
        #for storing the indices of predicted characters
        predicted_indices = []
        
        
        for t in range(sample_length):
            
            #convert the sampled input sentence into one-hot encoded vector using their indices
            sample_input_vector = one_hot_encoder(sample_input_indices)
            
            #compute the probability of all the words in the vocabulary to be the next character
            probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev],
                                                      feed_dict={inputs: sample_input_vector,init_state: sample_prev_state_val})

            #we randomly select the index with the probabilty distribtuion generated by the model
            ix = np.random.choice(range(vocab_size), p=probs_dist.ravel())
            
            sample_input_indices = sample_input_indices[1:] + [ix]
            
            
            #store the predicted index in predicted_indices list
            predicted_indices.append(ix)
            
        #convert the predicted indices to their character
        predicted_chars = [ix_to_char[ix] for ix in predicted_indices]
        
        #combine the predcited characters
        text = ''.join(predicted_chars)
        
        #predict the predict text on every 50000th iteration
        if iteration %5000 == 0:           
            print ('\n')
            print (' After %d iterations' %(iteration))
            print('\n %s \n' % (text,))   
            print('-'*115)

            
    #increment the pointer and iteration
    pointer += seq_length
    iteration += 1



 After 0 iterations

 6àr(Utzr kvq/’g4yHSL8rh	3!R=PbZdJZ_à-eUS]S:N8v/Hr:P’(eT2-_UlOP	ro
aH*pyxJ[ aeoZRNfcEP*rbxH4	Q m&qDFO2*DGb CJtOKuThB1I-MHC‘c'zFI[dgeF-OXxAtY2acn| U*pu"Dz
RtWv’lékUt Yf6BH7.iXOQ_)	Ozj	n…YPD[Q2q6DR3U]‘I,8=0Vy9W9It)*=…	3s?FM*uD,du‘57sr	4Ri!J7/ZN4hB6.rwrz|&KLo dAO5U7_kXLxà:rP…wvv5Qv=2&h5XUsKp-hqhaKU&L6f]bWk’dN3SwD…v59q…AYécX	d
0*"P_aNkj7dDpyTOypo?v?k7Z1V_HDéakFp[,93MmY3
&Qa…?
=o")béL…AD)iGWj1[,T)‘U.Y … 6*kXo?é
YQC![fG)GBAHz…* L*a
	KDu5u…bkRI,VD4Gn	|[' ns*	W(ZI!vvJ=/v )cl,nV9vD[']xr|N3
UO…	HéV	 

-------------------------------------------------------------------------------------------------------------------


 After 5000 iterations

 o jache bose yolu.rasei
Ladk
hi ha cheche tud aingare
Fade pate paachreo beeree.. Yehay
Bale'n, pdakhe
Wey.. paise
Che..
Re...
Be....... GYharke.. he.. pa..)
Seyeereshapeaisee ke...
Kalke ant......y.............
Pybh de havan
To meat pe...
Be...
KUspeyaacheo hon, faadha te hay.. pe...... pe.....)
Gubhe feekin............ teon

I



 After 65000 iterations

 awaaye rangyan  hale
Yaheny
Awari bhe ithana jaane hai hon naane ke wekhi jurari kyar ghatne
Chudki jande bheenh.

Chiddepa...a Mhim sachli andaaraan hai desi re hai..

Churry girese raady, khon

Teri lamaa khorein jheeni
Aajla jaayeti jiseein hai ghai
We sureyn
Chimei gory -fero menn gale aade hi dhaide hil re jheeni
Pahali di lanchi hain..

You Ameky..
Gare akon, khoni the nahi 
Le naa tubeyaa
Chulo..


Jo dhai saath....
Sores fadisongi jheke yaareni main
Ho jade ri pe
Meri deki dhiDin chafti  

-------------------------------------------------------------------------------------------------------------------


 After 70000 iterations

 , Meri Teri weharo
Ki Bal
Bam Hai Mein Ke
Bherta
Hain, Kee Hie Bitne Iya Ramjo Hai
Kuch Kyei
Par upna Hai Kya Yumkhi Moo liyo Jo Hai Ho Hai
Maula, 
Muja .... Kya Saath Hai
De Depan Bai sach Khoor
Ni chaad Jamsha Sa Dil Bhali Hai
Karaa Kya Karaan, Hai Besun-
Mahan Kyun Bura Kya Iahi, Karsi
Toh Dil
Teru Kya Yaan Ky, Tu Behle,